In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# Нерівність CHSH

*Оцінка використання: Дві хвилини на процесорі Heron r2 (ПРИМІТКА: Це лише оцінка. Ваш час виконання може відрізнятися.)*
## Загальна інформація
У цьому посібнику Ви виконаєте експеримент на квантовому комп'ютері, щоб продемонструвати порушення нерівності CHSH за допомогою примітиву Estimator.

Нерівність CHSH, названа на честь авторів Клаузера, Хорна, Шимоні та Холта, використовується для експериментального доведення теореми Белла (1969). Ця теорема стверджує, що теорії локальних прихованих змінних не можуть пояснити деякі наслідки заплутаності в квантовій механіці. Порушення нерівності CHSH використовується для демонстрації того, що квантова механіка несумісна з теоріями локальних прихованих змінних. Це важливий експеримент для розуміння основ квантової механіки.

Нобелівську премію з фізики 2022 року було присуджено Алену Аспе, Джону Клаузеру та Антону Цайлінгеру частково за їхню піонерську роботу в квантовій інформатиці, зокрема за експерименти з заплутаними фотонами, що демонструють порушення нерівностей Белла.
## Вимоги
Перед початком цього посібника переконайтеся, що у Вас встановлено наступне:

* Qiskit SDK v1.0 або новіший, з підтримкою [візуалізації](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 або новіший
## Налаштування

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Крок 1: Відображення класичних входів на квантову задачу
Для цього експерименту ми створимо заплутану пару, де будемо вимірювати кожен кубіт у двох різних базисах. Ми позначимо базиси для першого кубіта $A$ і $a$, а базиси для другого кубіта $B$ і $b$. Це дозволяє нам обчислити величину CHSH $S_1$:

$$
S_1 = A(B-b) + a(B+b).
$$

Кожна спостережувана величина дорівнює або $+1$, або $-1$. Очевидно, один з членів $B\pm b$ повинен дорівнювати $0$, а інший має бути $\pm 2$. Отже, $S_1 = \pm 2$. Середнє значення $S_1$ має задовольняти нерівність:

$$
|\langle S_1 \rangle|\leq 2.
$$

Розкладання $S_1$ через $A$, $a$, $B$ і $b$ дає:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Ви можете визначити іншу величину CHSH $S_2$:

$$
S_2 = A(B+b) - a(B-b),
$$

Це призводить до іншої нерівності:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Якщо квантову механіку можна описати теоріями локальних прихованих змінних, попередні нерівності мають виконуватися. Однак, як демонструється в цьому посібнику, ці нерівності можуть бути порушені на квантовому комп'ютері. Отже, квантова механіка несумісна з теоріями локальних прихованих змінних.
Якщо Ви хочете дізнатися більше теорії, вивчіть [Заплутаність в дії](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) з Джоном Ватрусом.
Ви створите заплутану пару між двома кубітами в квантовому комп'ютері, створивши стан Белла $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$. Використовуючи примітив Estimator, Ви можете безпосередньо отримати необхідні очікувані значення ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ і $\langle ab \rangle$) для обчислення очікуваних значень двох величин CHSH $\langle S_1\rangle$ і $\langle S_2\rangle$. До введення примітиву Estimator Вам довелося б конструювати очікувані значення з результатів вимірювань.

Ви будете вимірювати другий кубіт у базисах $Z$ і $X$. Перший кубіт також буде вимірюватися в ортогональних базисах, але під кутом відносно другого кубіта, який ми будемо змінювати від $0$ до $2\pi$. Як Ви побачите, примітив Estimator значно спрощує виконання параметризованих схем. Замість створення серії схем CHSH, Вам потрібно створити лише *одну* схему CHSH з параметром, що визначає кут вимірювання, та серію значень фази для параметра.

Нарешті, Ви проаналізуєте результати та побудуєте їх графік відносно кута вимірювання. Ви побачите, що для певного діапазону кутів вимірювання очікувані значення величин CHSH $|\langle S_1\rangle| > 2$ або $|\langle S_2\rangle| > 2$, що демонструє порушення нерівності CHSH.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### Створення параметризованої схеми CHSH
Спочатку ми напишемо схему з параметром $\theta$, який ми назвемо `theta`. [Примітив `Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) може значно спростити побудову схеми та аналіз виводу, безпосередньо надаючи очікувані значення спостережуваних величин. Багато цікавих задач, особливо для застосувань найближчого майбутнього на зашумлених системах, можна сформулювати в термінах очікуваних значень. Примітив `Estimator` (V2) може автоматично змінювати базис вимірювання на основі наданої спостережуваної величини.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### Створення списку значень фази для подальшого призначення
Після створення параметризованої схеми CHSH Ви створите список значень фази, які будуть призначені схемі на наступному кроці. Ви можете використовувати наступний код для створення списку з 21 значення фази в діапазоні від $0$ до $2 \pi$ з рівномірним інтервалом, тобто $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Спостережувані величини
Тепер нам потрібні спостережувані величини, з яких можна обчислити очікувані значення. У нашому випадку ми розглядаємо ортогональні базиси для кожного кубіта, дозволяючи параметризованому $Y-$обертанню для першого кубіта майже безперервно змінювати базис вимірювання відносно базису другого кубіта. Тому ми виберемо спостережувані величини $ZZ$, $ZX$, $XZ$ і $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Крок 2: Оптимізація задачі для виконання на квантовому обладнанні

Щоб зменшити загальний час виконання завдання, примітиви V2 приймають лише схеми та спостережувані величини, що відповідають інструкціям та з'єднанню, підтримуваним цільовою системою (які називаються схемами та спостережуваними величинами архітектури набору інструкцій (ISA)).

### ISA-схема

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA-спостережувані величини

Аналогічно, нам потрібно перетворити спостережувані величини, щоб зробити їх сумісними з бекендом перед запуском завдань з [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run). Ми можемо виконати перетворення за допомогою методу `apply_layout` об'єкта `SparsePauliOp`.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Крок 3: Виконання за допомогою примітивів Qiskit
Щоб виконати весь експеримент одним викликом [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2).
Ми можемо створити [примітив `Estimator` Qiskit Runtime](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2) для обчислення наших очікуваних значень. Метод `EstimatorV2.run()` приймає ітерований об'єкт `примітивних уніфікованих блоків (PUB)`. Кожен PUB — це ітерований об'єкт у форматі `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)